In [34]:
import numpy as np 
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch import optim
import torchvision.models as models
import cv2
import csv
import sys
import os
#from PIL import Image
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision.transforms import Compose, ToTensor
from tqdm.notebook import tqdm
from torch.utils.data.sampler import SubsetRandomSampler
from torch.autograd import Function
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import PIL.Image
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from torchvision.models.resnet import ResNet, BasicBlock
import inspect
#from tqdm.autonotebook import tqdm
#from tqdm import tqdm

In [23]:
class finalResNet(ResNet):
    def __init__(self):
        super(finalResNet, self).__init__(BasicBlock, [2, 2, 2, 2], num_classes=10)
        self.conv1 = torch.nn.Conv2d(3, 64, 
            kernel_size=(5, 5), 
            stride=(2, 2), 
            padding=(2, 2), bias=False)
        
    def forward(self, x):
        return torch.softmax(
            super(finalResNet, self).forward(x), dim=-1)
model = finalResNet()
print(model)

finalResNet(
  (conv1): Conv2d(3, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=Tru

In [24]:
# detect is gpu available.
use_gpu = torch.cuda.is_available()
if use_gpu:
    print("cuda")
    device =torch.device('cuda:0')
else:
    print("cpu")
    device =torch.device("cpu")
    
data_transforms =   transforms.Compose([
                    transforms.Resize((224, 224)),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                ])
    
class Image_data(Dataset):
    def __init__(self,label,data,data_transforms):
        self.data_path = data
        self.label_path = label
        self.data_transforms = data_transforms
    def __getitem__(self,index):

        trainX = np.load(self.data_path)
        trainY = np.load(self.label_path)
        image = PIL.Image.fromarray(np.uint8(trainX[index]))

        
        #trainX[index] = np.transpose(trainX[index], ( 2, 0, 1))
        #trainX = torch.Tensor(trainX)
        trainY = torch.Tensor(trainY)
        
        return data_transforms(image),trainY[index]

    def __len__(self):
        return len(np.load(self.label_path))

train_data = Image_data('./trainY.npy','./trainX.npy',data_transforms)

########################################################################################################################################################
########################################################################################################################################################
########################################################################################################################################################
########################################################################################################################################################
batch_size = 32
validation_split = .1
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(train_data)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = DataLoader(train_data, batch_size=batch_size,sampler=train_sampler)
val_loader = DataLoader(train_data, batch_size=batch_size,sampler=valid_sampler)

cuda


In [30]:

def calculate_metric(metric_fn, true_y, pred_y):
    if "average" in inspect.getfullargspec(metric_fn).args:
        return metric_fn(true_y, pred_y, average="macro")
    else:
        return metric_fn(true_y, pred_y)
    
def print_scores(p, r, f1, a, batch_size):
    for name, scores in zip(("precision", "recall", "F1", "accuracy"), (p, r, f1, a)):
        print(f"\t{name.rjust(14, ' ')}: {sum(scores)/batch_size:.4f}")

In [36]:
model = model.to(device)
start_ts = time.time()
losses = []
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adadelta(model.parameters())
epochs = 500
batches = len(train_loader)
val_batches = len(val_loader)

# training loop + eval loop
for epoch in range(epochs):
    total_loss = 0
    progress = tqdm(enumerate(train_loader), desc="Loss: ", total=batches)
    model.train()
    
    for i, data in progress:
        X, y = data[0].to(device), data[1].to(device)
        y = y.long()  
        
        model.zero_grad()
        outputs = model(X)
        loss = loss_function(outputs, y)

        loss.backward()
        optimizer.step()
        current_loss = loss.item()
        total_loss += current_loss
        progress.set_description("Loss: {:.4f}".format(total_loss/(i+1)))
        
    torch.cuda.empty_cache()
    
    val_losses = 0
    precision, recall, f1, accuracy = [], [], [], []
    
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            X, y = data[0].to(device), data[1].to(device)
            y = y.long()
            outputs = model(X)
            val_losses += loss_function(outputs, y)

            predicted_classes = torch.max(outputs, 1)[1]
            
            for acc, metric in zip((precision, recall, f1, accuracy), 
                                   (precision_score, recall_score, f1_score, accuracy_score)):
                acc.append(
                    calculate_metric(metric, y.cpu(), predicted_classes.cpu())
                )
        
        
        
    print(f"Epoch {epoch+1}/{epochs}, training loss: {total_loss/batches}, validation loss: {val_losses/val_batches}")
    print_scores(precision, recall, f1, accuracy, val_batches)
    losses.append(total_loss/batches)
print(losses)
print(f"Training time: {time.time()-start_ts}s")


Epoch 1/500, training loss: 2.085938588101813, validation loss: 2.07562518119812
	     precision: 0.3065
	        recall: 0.3914
	            F1: 0.3121
	      accuracy: 0.3812



Epoch 2/500, training loss: 2.0647676100967622, validation loss: 2.0714166164398193
	     precision: 0.3536
	        recall: 0.3779
	            F1: 0.3337
	      accuracy: 0.3805



Epoch 3/500, training loss: 2.059797730006225, validation loss: 2.0811383724212646
	     precision: 0.2893
	        recall: 0.3745
	            F1: 0.2892
	      accuracy: 0.3680



Epoch 4/500, training loss: 2.0603888821094594, validation loss: 2.038602828979492
	     precision: 0.3605
	        recall: 0.4128
	            F1: 0.3559
	      accuracy: 0.4141



Epoch 5/500, training loss: 2.041185260664487, validation loss: 2.0789411067962646
	     precision: 0.2938
	        recall: 0.3634
	            F1: 0.2944
	      accuracy: 0.3754


...


Epoch 496/500, training loss: 1.4689234131616904, validation loss: 1.7283539772033691
	     precision: 0.6979
	        recall: 0.7110
	            F1: 0.6780
	      accuracy: 0.7449



Epoch 497/500, training loss: 1.4689932486689683, validation loss: 1.7377359867095947
	     precision: 0.7180
	        recall: 0.7308
	            F1: 0.6950
	      accuracy: 0.7191



Epoch 498/500, training loss: 1.4691279078206272, validation loss: 1.737043857574463
	     precision: 0.7270
	        recall: 0.7322
	            F1: 0.7002
	      accuracy: 0.7281



Epoch 499/500, training loss: 1.468912751962107, validation loss: 1.7221908569335938
	     precision: 0.7313
	        recall: 0.7343
	            F1: 0.7003
	      accuracy: 0.7418



Epoch 500/500, training loss: 1.4688795086339856, validation loss: 1.7337818145751953
	     precision: 0.7167
	        recall: 0.7166
	            F1: 0.6929
	      accuracy: 0.7293
[2.085938588101813, 2.0647676100967622, 2.059797730006225, 2.0603888821094594, 2.041185260664487, 2.0371751878278475, 2.017956902794804, 2.0006014083294157, 2.0034904124888966, 1.9899267516237624, 1.96793336563922, 1.9523255368496508, 1.9398703110133502, 1.9341653010523912, 1.9286252775936261, 1.9045955695159047, 1.906237447515447, 1.8948739166800856, 1.8939499838132385, 1.872069921899349, 1.8615713525325694, 1.8527468104734488, 1.8347486291371338, 1.8349596676251567, 1.8184505827883457, 1.8195294877316088, 1.8050639823818884, 1.7987674948171521, 1.7920967763197337, 1.7745278202895578, 1.7695112879394639, 1.7667329522734838, 1.7580647333293942, 1.7519215117109583, 1.7393060499894704, 1.7313948768250487, 1.7250526120476688, 1.7198734452538456, 1.7049806523830333, 1.6941168139166867, 1.687820111606138, 1.684

In [37]:
torch.save(model,'trained_models/torchvision_model.pt')

/home/t-rex/anaconda3/envs/ml2019fall_hw1/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type finalResNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [40]:

test = torch.load('trained_models/torchvision_model.pt')

In [41]:
print(test)

finalResNet(
  (conv1): Conv2d(3, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=Tru